# 📊 Kaleidoscope Logic Analysis Notebook

Statistical and analytical exploration of kaleidoscope sequences.

## Analysis Tools
1. Phase statistics and distributions
2. Pattern complexity metrics
3. Fourier analysis of sequences
4. Correlation studies
5. Machine learning analysis

## Requirements
```bash
pip install numpy pandas matplotlib seaborn scipy scikit-learn
```

In [ ]:
# Cell 1: Setup and imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats, fft
from scipy.spatial import distance
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Analysis libraries imported")

## 1. Load and Explore Data

In [ ]:
# Cell 2: Load sequence data
import json
import os

def load_sequence_data(sequence_name='main'):
    """Load sequence metadata and calculate statistics"""
    
    metadata_path = f"../outputs/metadata/{sequence_name}_metadata.json"
    
    if os.path.exists(metadata_path):
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        
        # Convert to DataFrame
        df = pd.DataFrame(metadata['frames_data'])
        
        print(f"📊 Loaded {sequence_name} sequence: {len(df)} frames")
        print(f"  Phase range: {df['phase_diff'].min():.3f} - {df['phase_diff'].max():.3f} rad")
        print(f"  Δθ per frame: {metadata.get('delta_left', 'N/A')} (L), {metadata.get('delta_right', 'N/A')} (R)")
        
        return df, metadata
    else:
        print(f"⚠️  Metadata not found: {metadata_path}")
        print("Generating synthetic data for analysis...")
        
        # Generate synthetic data
        frames = 25
        delta_left = 0.08
        delta_right = 0.095
        
        data = []
        left_phase = 0.0
        right_phase = 0.39
        
        for i in range(frames):
            data.append({
                'frame': i,
                'left_angle': left_phase,
                'right_angle': right_phase,
                'phase_diff': right_phase - left_phase
            })
            left_phase = (left_phase + delta_left) % (2*np.pi)
            right_phase = (right_phase + delta_right) % (2*np.pi)
        
        df = pd.DataFrame(data)
        metadata = {
            'sequence_name': sequence_name + '_synthetic',
            'frames': frames,
            'delta_left': delta_left,
            'delta_right': delta_right
        }
        
        return df, metadata

# Load main sequence
df_main, meta_main = load_sequence_data('main')
df_main.head()

## 2. Statistical Analysis

In [ ]:
# Cell 3: Comprehensive statistical analysis
def analyze_sequence_statistics(df):
    """Perform comprehensive statistical analysis"""
    
    print("📈 SEQUENCE STATISTICS")
    print("=" * 50)
    
    # Basic statistics
    print("\nBasic Statistics:")
    print(f"  Total frames: {len(df)}")
    print(f"  Left phase range: {df['left_angle'].min():.3f} - {df['left_angle'].max():.3f} rad")
    print(f"  Right phase range: {df['right_angle'].min():.3f} - {df['right_angle'].max():.3f} rad")
    print(f"  Phase difference range: {df['phase_diff'].min():.3f} - {df['phase_diff'].max():.3f} rad")
    
    # Central tendencies
    print("\nCentral Tendencies:")
    print(f"  Mean phase diff: {df['phase_diff'].mean():.3f} rad")
    print(f"  Median phase diff: {df['phase_diff'].median():.3f} rad")
    print(f"  Mode phase diff: {stats.mode(df['phase_diff']).mode[0]:.3f} rad")
    
    # Dispersion
    print("\nDispersion Measures:")
    print(f"  Std deviation: {df['phase_diff'].std():.3f} rad")
    print(f"  Variance: {df['phase_diff'].var():.3f} rad²")
    print(f"  Range: {df['phase_diff'].max() - df['phase_diff'].min():.3f} rad")
    print(f"  IQR: {df['phase_diff'].quantile(0.75) - df['phase_diff'].quantile(0.25):.3f} rad")
    
    # Shape
    print("\nDistribution Shape:")
    print(f"  Skewness: {df['phase_diff'].skew():.3f}")
    print(f"  Kurtosis: {df['phase_diff'].kurtosis():.3f}")
    
    # Hypothesis testing
    print("\nHypothesis Tests:")
    
    # Test for linear trend
    from scipy.stats import linregress
    slope, intercept, r_value, p_value, std_err = linregress(df['frame'], df['phase_diff'])
    print(f"  Linear trend test: r = {r_value:.3f}, p = {p_value:.3e}")
    
    # Test for normality
    from scipy.stats import shapiro
    stat, p = shapiro(df['phase_diff'])
    print(f"  Normality test (Shapiro-Wilk): W = {stat:.3f}, p = {p:.3e}")
    
    # Test for stationarity (simplified)
    from scipy.stats import kendalltau
    tau, p_tau = kendalltau(df['frame'], df['phase_diff'])
    print(f"  Stationarity test (Kendall's tau): τ = {tau:.3f}, p = {p_tau:.3e}")
    
    return {
        'basic': {
            'frames': len(df),
            'phase_range': (df['phase_diff'].min(), df['phase_diff'].max())
        },
        'central': {
            'mean': df['phase_diff'].mean(),
            'median': df['phase_diff'].median(),
            'std': df['phase_diff'].std()
        },
        'tests': {
            'linear_r': r_value,
            'linear_p': p_value,
            'normality_p': p,
            'stationarity_tau': tau
        }
    }

# Run analysis
stats_results = analyze_sequence_statistics(df_main)

## 3. Visualization of Statistics

In [ ]:
# Cell 4: Create comprehensive statistical visualizations
def create_statistical_visualizations(df, sequence_name='main'):
    """Create all statistical visualizations"""
    
    fig = plt.figure(figsize=(15, 10))
    
    # 1. Distribution histogram with KDE
    ax1 = plt.subplot(3, 3, 1)
    sns.histplot(df['phase_diff'], kde=True, ax=ax1, color='skyblue', edgecolor='black')
    ax1.axvline(df['phase_diff'].mean(), color='red', linestyle='--', label=f'Mean: {df["phase_diff"].mean():.3f}')
    ax1.axvline(df['phase_diff'].median(), color='green', linestyle='--', label=f'Median: {df["phase_diff"].median():.3f}')
    ax1.set_xlabel('Phase Difference Δθ (rad)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Distribution of Phase Differences')
    ax1.legend()
    
    # 2. Box plot
    ax2 = plt.subplot(3, 3, 2)
    sns.boxplot(y=df['phase_diff'], ax=ax2, color='lightgreen')
    ax2.set_ylabel('Phase Difference Δθ (rad)')
    ax2.set_title('Box Plot of Phase Differences')
    
    # 3. QQ plot for normality
    ax3 = plt.subplot(3, 3, 3)
    stats.probplot(df['phase_diff'], dist="norm", plot=ax3)
    ax3.set_title('Q-Q Plot for Normality')
    
    # 4. Time series decomposition
    ax4 = plt.subplot(3, 3, 4)
    ax4.plot(df['frame'], df['phase_diff'], 'b-', linewidth=2, label='Phase Difference')
    
    # Add trend line
    z = np.polyfit(df['frame'], df['phase_diff'], 1)
    p = np.poly1d(z)
    ax4.plot(df['frame'], p(df['frame']), 'r--', linewidth=2, label=f'Trend: {z[0]:.3f}x + {z[1]:.3f}')
    
    ax4.set_xlabel('Frame')
    ax4.set_ylabel('Δθ (rad)')
    ax4.set_title('Time Series with Linear Trend')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Autocorrelation
    ax5 = plt.subplot(3, 3, 5)
    from pandas.plotting import autocorrelation_plot
    autocorrelation_plot(df['phase_diff'], ax=ax5)
    ax5.set_title('Autocorrelation Function')
    
    # 6. Correlation heatmap
    ax6 = plt.subplot(3, 3, 6)
    correlation = df[['left_angle', 'right_angle', 'phase_diff']].corr()
    sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, ax=ax6, square=True)
    ax6.set_title('Correlation Matrix')
    
    # 7. Phase space with density
    ax7 = plt.subplot(3, 3, 7)
    scatter = ax7.scatter(df['left_angle'], df['right_angle'], 
                         c=df['frame'], cmap='viridis', s=50, alpha=0.7)
    ax7.plot(df['left_angle'], df['right_angle'], 'gray', alpha=0.3, linewidth=1)
    ax7.set_xlabel('Left Phase θL (rad)')
    ax7.set_ylabel('Right Phase θR (rad)')
    ax7.set_title('Phase Space with Frame Coloring')
    ax7.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax7, label='Frame')
    
    # 8. Rolling statistics
    ax8 = plt.subplot(3, 3, 8)
    window = min(5, len(df) // 4)
    rolling_mean = df['phase_diff'].rolling(window=window).mean()
    rolling_std = df['phase_diff'].rolling(window=window).std()
    
    ax8.plot(df['frame'], df['phase_diff'], 'b-', alpha=0.3, label='Original')
    ax8.plot(df['frame'], rolling_mean, 'r-', linewidth=2, label=f'Rolling Mean (w={window})')
    ax8.fill_between(df['frame'], 
                     rolling_mean - rolling_std, 
                     rolling_mean + rolling_std, 
                     alpha=0.2, color='red', label='Rolling Std')
    
    ax8.set_xlabel('Frame')
    ax8.set_ylabel('Δθ (rad)')
    ax8.set_title('Rolling Statistics')
    ax8.legend()
    ax8.grid(True, alpha=0.3)
    
    # 9. Statistical summary
    ax9 = plt.subplot(3, 3, 9)
    ax9.axis('off')
    
    stats_text = (
        f"STATISTICAL SUMMARY\n"
        f"Sequence: {sequence_name}\n"
        f"Frames: {len(df)}\n"
        f"\n"
        f"Phase Difference Δθ\n"
        f"Mean: {df['phase_diff'].mean():.3f}\n"
        f"Std: {df['phase_diff'].std():.3f}\n"
        f"Min: {df['phase_diff'].min():.3f}\n"
        f"Max: {df['phase_diff'].max():.3f}\n"
        f"\n"
        f"Trend Analysis\n"
        f"Slope: {stats_results['tests']['linear_r']:.3f}\n"
        f"p-value: {stats_results['tests']['linear_p']:.3e}\n"
        f"Normality: {stats_results['tests']['normality_p']:.3e}"
    )
    
    ax9.text(0.1, 0.95, stats_text, transform=ax9.transAxes,
             fontsize=9, fontfamily='monospace',
             verticalalignment='top',
             bbox=dict(boxstyle="round,pad=0.5", facecolor='lightyellow', alpha=0.8))
    
    plt.suptitle(f'Comprehensive Statistical Analysis: {sequence_name.upper()} Sequence', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

# Create visualizations
create_statistical_visualizations(df_main, 'main')

## 4. Fourier and Spectral Analysis

In [ ]:
# Cell 5: Fourier and spectral analysis
def analyze_spectral_properties(df):
    """Perform Fourier and spectral analysis"""
    
    signal = df['phase_diff'].values
    N = len(signal)
    T = 1.0  # Sampling interval (1 frame)
    
    # Compute FFT
    yf = fft.fft(signal)
    xf = fft.fftfreq(N, T)[:N//2]
    
    # Power spectrum
    power_spectrum = 2.0/N * np.abs(yf[0:N//2])
    
    # Find dominant frequencies
    dominant_idx = np.argmax(power_spectrum[1:]) + 1  # Skip DC component
    dominant_freq = xf[dominant_idx]
    dominant_power = power_spectrum[dominant_idx]
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # 1. Original signal
    axes[0, 0].plot(df['frame'], signal, 'b-', linewidth=2)
    axes[0, 0].set_xlabel('Frame')
    axes[0, 0].set_ylabel('Phase Difference Δθ')
    axes[0, 0].set_title('Original Signal')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Power spectrum
    axes[0, 1].plot(xf[1:], power_spectrum[1:], 'r-', linewidth=2)
    axes[0, 1].axvline(dominant_freq, color='green', linestyle='--', 
                      label=f'Dominant: {dominant_freq:.3f} frame⁻¹')
    axes[0, 1].set_xlabel('Frequency (frame⁻¹)')
    axes[0, 1].set_ylabel('Power')
    axes[0, 1].set_title('Power Spectrum')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Spectrogram
    from scipy.signal import spectrogram
    f, t, Sxx = spectrogram(signal, fs=1.0, nperseg=min(8, N//2))
    im = axes[1, 0].pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud', cmap='viridis')
    axes[1, 0].set_xlabel('Frame')
    axes[1, 0].set_ylabel('Frequency (frame⁻¹)')
    axes[1, 0].set_title('Spectrogram')
    plt.colorbar(im, ax=axes[1, 0], label='Power (dB)')
    
    # 4. Cumulative periodogram
    axes[1, 1].plot(xf[1:], np.cumsum(power_spectrum[1:]), 'purple', linewidth=2)
    axes[1, 1].set_xlabel('Frequency (frame⁻¹)')
    axes[1, 1].set_ylabel('Cumulative Power')
    axes[1, 1].set_title('Cumulative Periodogram')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.suptitle('Spectral Analysis of Phase Evolution', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("📡 SPECTRAL ANALYSIS RESULTS")
    print("=" * 40)
    print(f"Signal length: {N} frames")
    print(f"Sampling frequency: {1/T} frame⁻¹")
    print(f"Nyquist frequency: {0.5/T} frame⁻¹")
    print(f"Dominant frequency: {dominant_freq:.3f} frame⁻¹")
    print(f"Dominant period: {1/dominant_freq:.1f} frames" if dominant_freq > 0 else "No dominant frequency")
    print(f"Total power: {np.sum(power_spectrum):.3f}")
    print(f"Power in dominant frequency: {dominant_power:.3f} ({dominant_power/np.sum(power_spectrum)*100:.1f}% of total)")
    
    return {
        'dominant_frequency': dominant_freq,
        'dominant_power': dominant_power,
        'total_power': np.sum(power_spectrum),
        'power_spectrum': power_spectrum,
        'frequencies': xf
    }

# Run spectral analysis
spectral_results = analyze_spectral_properties(df_main)

## 5. Machine Learning Analysis

In [ ]:
# Cell 6: Machine learning analysis
def analyze_with_ml(df):
    """Apply machine learning techniques"""
    
    try:
        from sklearn.ensemble import IsolationForest
        from sklearn.cluster import KMeans, DBSCAN
        from sklearn.decomposition import PCA
        from sklearn.preprocessing import StandardScaler
        from sklearn.metrics import silhouette_score
        
        print("🤖 Applying Machine Learning Analysis...")
        
        # Prepare features
        X = df[['left_angle', 'right_angle', 'phase_diff']].values
        
        # Standardize
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        fig, axes = plt.subplots(2, 3, figsize=(15, 8))
        
        # 1. PCA visualization
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X_scaled)
        
        axes[0, 0].scatter(X_pca[:, 0], X_pca[:, 1], c=df['frame'], cmap='viridis', s=50, alpha=0.7)
        axes[0, 0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
        axes[0, 0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
        axes[0, 0].set_title('PCA of Phase Space')
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. K-means clustering
        kmeans = KMeans(n_clusters=min(4, len(df)//3), random_state=42)
        clusters = kmeans.fit_predict(X_scaled)
        
        axes[0, 1].scatter(df['left_angle'], df['right_angle'], c=clusters, cmap='tab10', s=50, alpha=0.7)
        axes[0, 1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
                          marker='X', s=200, c='red', label='Centroids')
        axes[0, 1].set_xlabel('Left Phase θL')
        axes[0, 1].set_ylabel('Right Phase θR')
        axes[0, 1].set_title(f'K-means Clustering (k={kmeans.n_clusters})')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # 3. Anomaly detection
        iso_forest = IsolationForest(contamination=0.1, random_state=42)
        anomalies = iso_forest.fit_predict(X_scaled)
        
        colors = ['blue' if a == 1 else 'red' for a in anomalies]
        axes[0, 2].scatter(df['frame'], df['phase_diff'], c=colors, s=50, alpha=0.7)
        axes[0, 2].set_xlabel('Frame')
        axes[0, 2].set_ylabel('Phase Difference Δθ')
        axes[0, 2].set_title('Anomaly Detection (Isolation Forest)')
        axes[0, 2].grid(True, alpha=0.3)
        
        # 4. Silhouette analysis
        silhouette_scores = []
        k_range = range(2, min(8, len(df)//2))
        
        for k in k_range:
            kmeans_temp = KMeans(n_clusters=k, random_state=42)
            clusters_temp = kmeans_temp.fit_predict(X_scaled)
            if len(np.unique(clusters_temp)) > 1:
                score = silhouette_score(X_scaled, clusters_temp)
                silhouette_scores.append(score)
            else:
                silhouette_scores.append(0)
        
        axes[1, 0].plot(list(k_range), silhouette_scores, 'o-', linewidth=2)
        axes[1, 0].set_xlabel('Number of clusters (k)')
        axes[1, 0].set_ylabel('Silhouette Score')
        axes[1, 0].set_title('Silhouette Analysis')
        axes[1, 0].grid(True, alpha=0.3)
        
        # 5. Feature importance
        feature_names = ['Left θ', 'Right θ', 'Δθ']
        feature_importance = np.abs(pca.components_[0])  # Use first PC loadings
        
        axes[1, 1].bar(feature_names, feature_importance, color='skyblue', edgecolor='black')
        axes[1, 1].set_ylabel('Importance (PC1 loadings)')
        axes[1, 1].set_title('Feature Importance')
        axes[1, 1].grid(True, alpha=0.3, axis='y')
        
        # 6. ML summary
        axes[1, 2].axis('off')
        
        ml_text = (
            f"MACHINE LEARNING SUMMARY\n"
            f"Samples: {len(df)}\n"
            f"Features: 3\n"
            f"\n"
            f"PCA Results\n"
            f"Variance explained: {pca.explained_variance_ratio_.sum()*100:.1f}%\n"
            f"PC1: {pca.explained_variance_ratio_[0]*100:.1f}%\n"
            f"PC2: {pca.explained_variance_ratio_[1]*100:.1f}%\n"
            f"\n"
            f"Clustering\n"
            f"Optimal clusters: {list(k_range)[np.argmax(silhouette_scores)]}\n"
            f"Best silhouette: {max(silhouette_scores):.3f}\n"
            f"\n"
            f"Anomalies: {(anomalies == -1).sum()} ({((anomalies == -1).sum()/len(df))*100:.1f}%)"
        )
        
        axes[1, 2].text(0.1, 0.95, ml_text, transform=axes[1, 2].transAxes,
                       fontsize=9, fontfamily='monospace',
                       verticalalignment='top',
                       bbox=dict(boxstyle="round,pad=0.5", facecolor='lightblue', alpha=0.8))
        
        plt.suptitle('Machine Learning Analysis of Kaleidoscope Sequences', 
                     fontsize=14, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
        
        print("✅ Machine learning analysis complete!")
        
        return {
            'pca_variance': pca.explained_variance_ratio_,
            'optimal_clusters': list(k_range)[np.argmax(silhouette_scores)],
            'silhouette_score': max(silhouette_scores),
            'anomaly_count': (anomalies == -1).sum()
        }
        
    except ImportError as e:
        print(f"⚠️  scikit-learn not installed: {e}")
        print("Install with: pip install scikit-learn")
        return None

# Run ML analysis
ml_results = analyze_with_ml(df_main)

## 6. Export Analysis Results

In [ ]:
# Cell 7: Export analysis results
def export_analysis_results(df, stats_results, spectral_results, ml_results, filename='analysis_results.json'):
    """Export all analysis results to JSON"""
    
    results = {
        'metadata': {
            'sequence_name': 'main',
            'frames': len(df),
            'analysis_date': pd.Timestamp.now().isoformat(),
            'analysis_type': 'comprehensive'
        },
        'statistics': stats_results,
        'spectral_analysis': {
            'dominant_frequency': float(spectral_results['dominant_frequency']),
            'dominant_power': float(spectral_results['dominant_power']),
            'total_power': float(spectral_results['total_power'])
        },
        'machine_learning': ml_results if ml_results else 'Not available',
        'raw_data_summary': {
            'left_phase_stats': {
                'mean': float(df['left_angle'].mean()),
                'std': float(df['left_angle'].std()),
                'min': float(df['left_angle'].min()),
                'max': float(df['left_angle'].max())
            },
            'phase_diff_stats': {
                'mean': float(df['phase_diff'].mean()),
                'std': float(df['phase_diff'].std()),
                'min': float(df['phase_diff'].min()),
                'max': float(df['phase_diff'].max()),
                'trend_slope': float(np.polyfit(df['frame'], df['phase_diff'], 1)[0])
            }
        }
    }
    
    # Save to JSON
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2, default=str)
    
    # Also save to CSV
    csv_filename = filename.replace('.json', '.csv')
    df.to_csv(csv_filename, index=False)
    
    print(f"✅ Analysis results exported:")
    print(f"   JSON: {filename}")
    print(f"   CSV: {csv_filename}")
    
    # Print summary
    print("\n📊 ANALYSIS SUMMARY")
    print("=" * 40)
    print(f"Sequence: main ({len(df)} frames)")
    print(f"Phase difference: {df['phase_diff'].mean():.3f} ± {df['phase_diff'].std():.3f} rad")
    print(f"Trend slope: {np.polyfit(df['frame'], df['phase_diff'], 1)[0]:.3e} rad/frame")
    if spectral_results:
        print(f"Dominant frequency: {spectral_results['dominant_frequency']:.3f} frame⁻¹")
    if ml_results:
        print(f"Optimal clusters: {ml_results['optimal_clusters']}")
        print(f"Silhouette score: {ml_results['silhouette_score']:.3f}")

# Export results
export_analysis_results(df_main, stats_results, spectral_results, ml_results, 
                       filename='../outputs/analysis/main_analysis_results.json')

---

## Analysis Complete!

This notebook has performed:

1. **Statistical analysis** of phase distributions
2. **Spectral analysis** with Fourier transforms
3. **Machine learning** clustering and anomaly detection
4. **Comprehensive visualization** of all results
5. **Data export** for further analysis

To analyze different sequences:
```python
df, meta = load_sequence_data('rotation')  # or 'quick'
```